In [1]:
import sys
import os

# Add the current directory to the Python path
sys.path.append(os.getcwd())

In [3]:
from dotenv import load_dotenv

load_dotenv("I:/11_DFKI_Hiwi/Work/01_Code/Graphusion/.env")

api_key = os.getenv("OPENAI_API_KEY")

In [5]:
import os
import json
from langchain_core.prompts import ChatPromptTemplate  
from models import KnowledgeGraphLLM
# from My_Work.Code.Main_Implementation.models import KnowledgeGraphLLM

In [7]:
# load sample data
data = json.load(open('I:/11_DFKI_Hiwi/Work/01_Code/Graphusion/outputs/sample/concept_abstracts_sample.json', 'r'))
relation_types = json.load(open('I:/11_DFKI_Hiwi/Work/01_Code/Graphusion/inputs/relation_types.json', 'r'))

In [9]:
concept_name = "question answering qa"
print('Sample concept:', concept_name)

# Joining abstracts for 1st concept with -- inbetween to make it a single string

abstracts = ' --  '.join(data[concept_name]['abstracts'])
print('Used abstracts:', abstracts)

Sample concept: question answering qa
Used abstracts: A crucial issue of current text generation models is that they often uncontrollably generate text that is factually inconsistent with inputs. Due to lack of annotated data, existing factual consistency metrics usually train evaluation models on synthetic texts or directly transfer from other related tasks, such as question answering (QA) and natural language inference (NLI).Bias in synthetic text or upstream tasks makes them perform poorly on text actually generated by language models, especially for general evaluation for various tasks. To alleviate this problem, we propose a weakly supervised framework named WeCheck that is directly trained on actual generated samples from language models with weakly annotated labels.WeCheck first utilizes a generative model to infer the factual labels of generated samples by aggregating weak labels from multiple resources.Next, we train a simple noise-aware classification model as the target metr

In [11]:
len(abstracts)

60305

In [13]:
# initialize the prompt template
prompt_template_txt = open("I:/11_DFKI_Hiwi/Work/01_Code/Graphusion/prompts/prompt_tpextraction.txt").read()
print(prompt_template_txt)

"""
### Follow the INSTRUCTION carefully:
You are an expert in **Computer Science** and **Natural Language Processing (NLP)**, tasked with constructing a **knowledge graph**. Given a **query concept: {concept}** and a **context** which is provided at the end, perform the following tasks:

#### **Step 1: Extract Relevant Concepts**
- Identify **fine-grained concepts** from the content that are relevant to the **query concept**.
- These concepts should be well-defined and typically found in:
  - **Lecture topics**
  - **Academic material sections**
  - **Wikipedia pages**
- Extracted concepts may include **subfields, techniques, tools, datasets, tasks, models, or evaluation metrics**.

#### **Step 2: Identify Relationships and Generate Triplets**
- Determine the **relationships** between the **query concept** and the **extracted concepts** using the **triplet format**: (head_concept, relation, tail_concept)

#### **Step 3: Use only and only the 7 predefined relation types, otherwise, you

In [14]:
prompt_template = ChatPromptTemplate.from_template(prompt_template_txt)

print('Prompt template', prompt_template)

Prompt template input_variables=['abstract', 'concept', 'relation_definitions'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['abstract', 'concept', 'relation_definitions'], template='"""\n### Follow the INSTRUCTION carefully:\nYou are an expert in **Computer Science** and **Natural Language Processing (NLP)**, tasked with constructing a **knowledge graph**. Given a **query concept: {concept}** and a **context** which is provided at the end, perform the following tasks:\n\n#### **Step 1: Extract Relevant Concepts**\n- Identify **fine-grained concepts** from the content that are relevant to the **query concept**.\n- These concepts should be well-defined and typically found in:\n  - **Lecture topics**\n  - **Academic material sections**\n  - **Wikipedia pages**\n- Extracted concepts may include **subfields, techniques, tools, datasets, tasks, models, or evaluation metrics**.\n\n#### **Step 2: Identify Relationships and Generate Triplets**\n- Determine the **

In [15]:
input_data = {"abstract": abstracts, "concept": [concept_name], "relation_definitions": relation_types}
print('Input data:', input_data)

Input data: {'abstract': 'A crucial issue of current text generation models is that they often uncontrollably generate text that is factually inconsistent with inputs. Due to lack of annotated data, existing factual consistency metrics usually train evaluation models on synthetic texts or directly transfer from other related tasks, such as question answering (QA) and natural language inference (NLI).Bias in synthetic text or upstream tasks makes them perform poorly on text actually generated by language models, especially for general evaluation for various tasks. To alleviate this problem, we propose a weakly supervised framework named WeCheck that is directly trained on actual generated samples from language models with weakly annotated labels.WeCheck first utilizes a generative model to infer the factual labels of generated samples by aggregating weak labels from multiple resources.Next, we train a simple noise-aware classification model as the target metric using the inferred weakly

In [16]:
# instantiate the prompt template
prompt = prompt_template.invoke(input_data)

In [17]:
print('Prompt', prompt)

Prompt messages=[HumanMessage(content='"""\n### Follow the INSTRUCTION carefully:\nYou are an expert in **Computer Science** and **Natural Language Processing (NLP)**, tasked with constructing a **knowledge graph**. Given a **query concept: [\'question answering qa\']** and a **context** which is provided at the end, perform the following tasks:\n\n#### **Step 1: Extract Relevant Concepts**\n- Identify **fine-grained concepts** from the content that are relevant to the **query concept**.\n- These concepts should be well-defined and typically found in:\n  - **Lecture topics**\n  - **Academic material sections**\n  - **Wikipedia pages**\n- Extracted concepts may include **subfields, techniques, tools, datasets, tasks, models, or evaluation metrics**.\n\n#### **Step 2: Identify Relationships and Generate Triplets**\n- Determine the **relationships** between the **query concept** and the **extracted concepts** using the **triplet format**: (head_concept, relation, tail_concept)\n\n#### **S

#### Testing Pipeline with OpenAI

In [23]:
llm = KnowledgeGraphLLM(model_name="gpt-4o", max_tokens=10000, api_key=api_key)

In [24]:
output_file = "I:/11_DFKI_Hiwi/Work/01_Code/Graphusion/outputs/sample/triples_sample.json"

In [25]:
import json
from tqdm import tqdm

with open(output_file, 'a') as output_stream:  # Open file once to avoid multiple open/close
    for concept_id, (concept_name, concept_data) in tqdm(enumerate(data.items()), total=len(data.items())): 
        print('Processing concept:', concept_name)

        # Ensure 'abstracts' key exists and is not empty
        abstracts = ' --  '.join(concept_data.get('abstracts', []))
        print('Printing first 100 chars of the abstracts:', abstracts[:100])

        input_data = {
            "abstract": abstracts,  # Limit to 10,000 chars
            "concept": [concept_name],
            "relation_definitions": relation_types
        }

        print('Input data:', input_data)

        # Invoke prompt and get response
        prompt = prompt_template.invoke(input_data)
        response = llm.invoke(prompt)
        print('Response:', response)

        extracted_relations = []
        
        if response and response.strip() != "None":  # Ensure response is valid
            try:
                response_json = json.loads(response)  # Parse JSON response
                for triple in response_json:
                    if triple['p'] in relation_types:  # Only consider known relation types
                        extracted_relations.append(triple['p'])
                        triple['id'] = concept_id
                        triple['concept'] = input_data['concept']
                        output_stream.write(json.dumps(triple) + '\n')
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON response: {e}")

print("Processing complete. Extracted relations saved to:", output_file)

  0%|          | 0/20 [00:00<?, ?it/s]

Processing concept: natural language explanation
Printing first 100 chars of the abstracts: Current state-of-the-art models for natural language understanding require a preprocessing step to c
Input data: {'abstract': 'Current state-of-the-art models for natural language understanding require a preprocessing step to convert raw text into discrete tokens. This process known as tokenization relies on a pre-built vocabulary of words or sub-word morphemes. This fixed vocabulary limits the model’s robustness to spelling errors and its capacity to adapt to new domains. In this work, we introduce a novel open-vocabulary language model that adopts a hierarchical two-level approach: one at the word level and another at the sequence level. Concretely, we design an intra-word module that uses a shallow Transformer architecture to learn word representations from their characters, and a deep inter-word Transformer module that contextualizes each word representation by attending to the entire word s

  5%|▌         | 1/20 [00:07<02:23,  7.55s/it]

Response: [{"s": "natural language explanation", "p": "Evaluate-for", "o": "model reasoning"}, {"s": "natural language explanation", "p": "Evaluate-for", "o": "classification performance"}, {"s": "natural language explanation", "p": "Used-for", "o": "improving classification"}, {"s": "natural language explanation", "p": "Used-for", "o": "justifying predictions"}, {"s": "natural language explanation", "p": "Evaluate-for", "o": "inconsistencies detection"}, {"s": "natural language explanation", "p": "Evaluate-for", "o": "gender-fairness"}, {"s": "natural language explanation", "p": "Evaluate-for", "o": "ambiguity resolution"}, {"s": "natural language explanation", "p": "Part-of", "o": "natural language processing"}, {"s": "natural language explanation", "p": "Hyponym-Of", "o": "text explanation"}, {"s": "natural language explanation", "p": "Compare", "o": "text-to-image disambiguation"}, {"s": "natural language explanation", "p": "Used-for", "o": "enhancing language representation"}, {"s

 10%|█         | 2/20 [00:14<02:14,  7.47s/it]

...

Response: [{"s": "relation extraction", "p": "Part-of", "o": "information extraction"}, {"s": "pre-trained language models", "p": "Used-for", "o": "relation extraction"}, {"s": "continual relation extraction", "p": "Used-for", "o": "learning constantly emerging relations"}, {"s": "biomedical relation extraction", "p": "Used-for", "o": "multi-class classification task"}, {"s": "Open Information Extraction", "p": "Used-for", "o": "pipelines of various NLP tasks"}, {"s": "Open Information Extraction", "p": "Part-of", "o": "information extraction"}, {"s": "document-level relation extraction", "p": "Part-of", "o": "relation extraction"}, {"s": "Event Argument Extraction", "p": "Used-for", "o": "improving model generalizability"}, {"s": "multimodal relation extraction", "p": "Used-for", "o": "identifying semantic relationships between entities"}, {"s": "relation extraction", "p": "Used-for", "o": "acquisition of relational facts"}]
Processing concept: dialogue system
Printing first 100 chars

 95%|█████████▌| 19/20 [03:08<00:11, 11.21s/it]

Response: [{"s": "dialogue system", "p": "Part-of", "o": "chatbot"}, {"s": "open-domain dialogue system", "p": "Hyponym-Of", "o": "dialogue system"}, {"s": "Task-Oriented Dialogue System", "p": "Hyponym-Of", "o": "dialogue system"}, {"s": "SafeConv", "p": "Used-for", "o": "conversational safety research"}, {"s": "DIONYSUS", "p": "Used-for", "o": "dialogue summarization"}, {"s": "MoralDial", "p": "Used-for", "o": "moral dialogue system training"}, {"s": "retrieval-based dialogue system", "p": "Hyponym-Of", "o": "dialogue system"}, {"s": "multi-label intent detection", "p": "Used-for", "o": "Task-Oriented Dialogue System"}, {"s": "ACCENT", "p": "Evaluate-for", "o": "event commonsense evaluation"}, {"s": "mBART", "p": "Used-for", "o": "cross-lingual transfer"}, {"s": "T5", "p": "Used-for", "o": "cross-lingual transfer"}, {"s": "NEUPSL DSI", "p": "Used-for", "o": "dialog structure induction"}, {"s": "ASAP", "p": "Used-for", "o": "user satisfaction estimation"}, {"s": "MAKER", "p": "Used-fo

100%|██████████| 20/20 [03:13<00:00,  9.69s/it]

Response: [{"s": "relation extraction docre", "p": "Evaluate-for", "o": "document-level relation extraction"}, {"s": "relation extraction docre", "p": "Part-of", "o": "Universal Information Extraction"}, {"s": "relation extraction docre", "p": "Used-for", "o": "knowledge graph construction"}, {"s": "relation extraction docre", "p": "Compare", "o": "multilingual relation extraction"}, {"s": "relation extraction docre", "p": "Used-for", "o": "multimodal relation extraction"}, {"s": "relation extraction docre", "p": "Is-a-Prerequisite-of", "o": "event temporal relation extraction"}, {"s": "relation extraction docre", "p": "Used-for", "o": "event detection and event-relation extraction tasks"}]
Processing complete. Extracted relations saved to: I:/11_DFKI_Hiwi/Work/01_Code/Graphusion/outputs/sample/triples_sample.json
